In [1]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

# The Prince and the Tiger

Once upon a time, there was a king who wanted to marry his daughter to a prince. He had it proclaimed throughout the land that he was seeking a husband for his daughter. One day, a prince came by to apply for the position.

Since the king did not want to marry his daughter to some dullard, he led the prince into a room with nine doors. The king told the prince that the princess was in one of the rooms, but that there were other rooms behind which hungry tigers were waiting. Some rooms were also empty. If the prince opened a door with a tiger behind it, it would probably be his last mistake.

The king went on to say that there were signs on all the doors with statements on them. These statements behave as follows:

* In the rooms where there is a tiger, the statement on the sign is false.
* In the room where the princess is, the statement is true.
* With regards to the the empty rooms, the situation is a bit more complicated, because there are two possibilities:
    - Either **all** the inscriptions on the empty rooms are true,
    - or **all** the inscriptions on the empty rooms are false.

The prince then read the inscriptions. These were as follows:

1. Room: The princess is in a room with an odd number.
   There is no tiger in the rooms with an even number.
2. Room: This room is empty.
3. Room: The inscription on Room No. 5 is true, the inscription on Room No. 7 is false,
   and there is a tiger in Room No. 3.
4. Room: The inscription on Room No. 1 is false, there is no tiger in Room No. 8,
   and the inscription on Room No. 9 is true.
5. Room: If the inscription on Room No. 2 or on Room No. 4 is true,
   then there is no tiger in Room No. 1.
6. Room: The inscription on Room No. 3 is false, the princess is in Room No. 2,
   and there is no tiger in Room No. 2.
7. Room: The princess is in Room No. 1 and the inscription on Room No. 5 is true.
8. Room: There is no tiger in this room and Room No. 9 is empty.
9. Room: Neither in this room nor in Room No. 1 is there a tiger, and moreover, the
   inscription on Room No. 6 is true.

## Setting up the Required Modules

We will use the parser for propositional logic which is implemented in the module <tt>propLogParser</tt>.

In [2]:
%%capture
%run Propositional-Logic-Parser.ipynb

We will also need a function that turns a formula given as a nested tuple into *conjunctive normal form*.  Therefore we import the module <tt>cnf</tt>.

In [3]:
%%capture
%run 04-CNF.ipynb

The function $\texttt{parseKNF}(s)$ takes a string $s$ that represents a formula from propositional logic, parses this string as a propositional formula and then turns this formula into a set of clauses.  We have used this function already when discussing the Zebra problem.

In [4]:
def parseKNF(s):
    nestedTuple = parse(s)
    Clauses     = normalize(nestedTuple)
    return Clauses

In [5]:
parseKNF('(p ∧ ¬q → r) ↔ ¬r ∨ p ∨ q')

{frozenset({('¬', 'p'), 'q', 'r'}), frozenset({('¬', 'r'), 'p', 'q'})}

Finally, we use the Davis-Putnam algorithm to find a solution for a given set of clauses.  This algorithm is provided by the module <tt>davisPutnam</tt>.

In [6]:
%%capture
%run 07-Davis-Putnam-JW.ipynb

## Auxilliary Functions

The functions defined below make it convenient to create the propositional variables $\texttt{Prinzessin<}i\texttt{>}$, $\texttt{Tiger<}i\texttt{>}$, and $\texttt{Zimmer<}i\texttt{>}$ for $i \in \{1,\cdots,n\}$.

In order to satisfy the following constraint:

  * Either **all** the inscriptions on the empty rooms are true,
  * or **all** the inscriptions on the empty rooms are false.

you need another propositional variable.

In [7]:
def P(i):
    "Return the string 'Pi'"
    return f'P{i}'

In [8]:
P(1)

'P1'

In [9]:
parseKNF(f'{P(1)} ∨ {P(2)}')

{frozenset({'P1', 'P2'})}

In [10]:
def T(i):
    "Return the string 'Ti'"
    return f'T{i}'

In [11]:
T(2)

'T2'

In [12]:
def Z(i): #true or false inscription
    "Return the string 'Zi'"
    return f'Z{i}'

In [13]:
Z(3)

'Z3'

Given a set of propositional variables $S$, the function $\texttt{atMostOne}(S)$ computes a set of clauses expressing the fact that at most one of the variables of $S$ is <tt>True</tt>.

In [14]:
def atMostOne(S): 
    return { frozenset({('¬',p), ('¬', q)}) for p in S
                                            for q in S 
                                            if  p != q 
           }

## Generating the set of Clauses describing the Problem

##### Below, you might need the following symbols: ¬ ∧ ∨ → ↔

The function $\texttt{computeClauses}$ computes the set of clauses that encode the given problem.

In [18]:
def computeClauses():
    # The princess has to be somewhere, i.e. there is a room containing the princess.
    Clauses  = {frozenset({P(i) for i in range(1, 9+1)})}
    # There is just one princess.
    Clauses |= atMostOne({P(i) for i in range(1,10)})
    for i in range(1, 9+1):
        # There is no room containing the princess as well as the tiger.
        #Clauses |= parseKNF(f'¬{P(i)} ∨ ¬{T(i)}')
        # In the room containing the princess, the label at the door is true.
        Clauses |= parseKNF(f'{P(i)} → {Z(i)}')
        # In thoses rooms where there are tigers, the label is false.
        Clauses |= parseKNF(f'{T(i)}  → ¬{Z(i)}')
        # Either all labels of empty rooms are true or all those labels are false.
        Clauses |= parseKNF(f'(¬{P(i)} ∧ ¬{T(i)} ∧ e) → {Z(i)}')         
        Clauses |= parseKNF(f'(¬{P(i)} ∧ ¬{T(i)} ∧ ¬e) → ¬{Z(i)}')
    # Room Nr.1: The princess is in a room with an odd room number.
    #            The rooms with even numbers do not have tigers.
    Clauses |= parseKNF('Z1 ↔ ((P1 ∨ P3 ∨ P5 ∨ P7 ∨ P9) ∧ ¬T2 ∧ ¬T4 ∧ ¬T6 ∧ ¬T8)')
    # Room Nr.2: This room is empty.
    Clauses |= parseKNF('Z2 ↔ (¬P2 ∧ ¬T2)')
    # Room Nr.3: The label at room number 5 is true, the label at room number 7 is false 
    #            and there is a tiger in room number 3
    Clauses |= parseKNF('Z3 ↔ (Z5 ∧ ¬Z7 ∧ T3)')
    # Room Nr.4: The label at room number 1 is false, there is no tiger in room number 8
    #            and the label at room number 9 is true.
    Clauses |= parseKNF('Z4 ↔ (¬Z1 ∧ ¬T8 ∧ Z9)')
    # Room Nr.5: If the label at room number 2 or room number 4 is true, 
    #            then there is no tiger in room number 1.
    Clauses |= parseKNF('Z5 ↔ ((Z2 ∨ Z4) →  ¬T1)')
    # Room Nr.6: The label on room number 3 is false, the princess is in room number 2
    #            and there is no tiger in room number 2.
    Clauses |= parseKNF('Z6 ↔ (¬Z3 ∧ P2 ∧ ¬T2)')
    # Room Nr.7: The princess is in room number 1 and the label of room number 5 is true.
    Clauses |= parseKNF('Z7 ↔ (P1 ∧ Z5)')
    # Room Nr.8: There is no tiger in this room and room number 9 is empty.
    Clauses |= parseKNF('Z8 ↔ (¬T8 ∧ ¬T9 ∧ ¬P9)')
    # Room Nr.9: Neither this room nor room number 1 has a tiger and 
    #            the label of room number 6 is true.
    Clauses |= parseKNF('Z9 ↔ (¬T9 ∧ ¬T1 ∧ Z6)')
    return Clauses

In [19]:
Clauses = computeClauses()
Clauses

{frozenset({('¬', 'P3'), ('¬', 'P8')}),
 frozenset({('¬', 'T3'), ('¬', 'Z5'), 'Z3', 'Z7'}),
 frozenset({('¬', 'Z3'), ('¬', 'Z7')}),
 frozenset({('¬', 'P5'), ('¬', 'P6')}),
 frozenset({('¬', 'e'), 'P7', 'T7', 'Z7'}),
 frozenset({('¬', 'Z6'), 'P6', 'T6', 'e'}),
 frozenset({('¬', 'P3'), 'T2', 'T4', 'T6', 'T8', 'Z1'}),
 frozenset({('¬', 'P4'), ('¬', 'P8')}),
 frozenset({('¬', 'T5'), ('¬', 'Z5')}),
 frozenset({('¬', 'P7'), ('¬', 'P8')}),
 frozenset({('¬', 'P2'), ('¬', 'P7')}),
 frozenset({('¬', 'P4'), 'Z4'}),
 frozenset({('¬', 'e'), 'P6', 'T6', 'Z6'}),
 frozenset({('¬', 'Z8'), 'P8', 'T8', 'e'}),
 frozenset({('¬', 'P1'), ('¬', 'Z5'), 'Z7'}),
 frozenset({('¬', 'P1'), ('¬', 'P6')}),
 frozenset({('¬', 'e'), 'P4', 'T4', 'Z4'}),
 frozenset({('¬', 'P4'), ('¬', 'P5')}),
 frozenset({('¬', 'P2'), ('¬', 'P5')}),
 frozenset({('¬', 'Z7'), 'P7', 'T7', 'e'}),
 frozenset({('¬', 'P5'), ('¬', 'P7')}),
 frozenset({('¬', 'P5'), ('¬', 'P8')}),
 frozenset({('¬', 'P9'), 'Z9'}),
 frozenset({'P9', 'T8', 'T9', 'Z8'}

There are 110 clauses.

In [20]:
len(Clauses)

110

Finally, we call the function <tt>solve</tt> from the module <tt>davisPutnam</tt> to solve the problem.

In [21]:
%%time
solution = solve(Clauses)

CPU times: user 4.59 ms, sys: 948 μs, total: 5.54 ms
Wall time: 5.05 ms


In [22]:
solution

{frozenset({'P5'}),
 frozenset({('¬', 'e')}),
 frozenset({('¬', 'P6')}),
 frozenset({('¬', 'P4')}),
 frozenset({('¬', 'P8')}),
 frozenset({('¬', 'P1')}),
 frozenset({('¬', 'P9')}),
 frozenset({('¬', 'Z6')}),
 frozenset({('¬', 'T5')}),
 frozenset({('¬', 'Z2')}),
 frozenset({('¬', 'Z8')}),
 frozenset({('¬', 'P7')}),
 frozenset({('¬', 'Z9')}),
 frozenset({('¬', 'Z7')}),
 frozenset({('¬', 'Z1')}),
 frozenset({('¬', 'T3')}),
 frozenset({('¬', 'P2')}),
 frozenset({('¬', 'Z4')}),
 frozenset({'Z5'}),
 frozenset({'T8'}),
 frozenset({('¬', 'Z3')}),
 frozenset({'T2'}),
 frozenset({('¬', 'P3')})}

The function $\texttt{getSolution}(S)$ takes a set of unit clauses representing the solution of the problem and returns the room where the princess is located.

In [23]:
def getSolution(S):
    "Print only the positive literals from the set S."
    for Unit in S:
        for l in Unit:
            if isinstance(l, str) and l[0] == 'P':
                return l

We print the solution.

In [24]:
princess = getSolution(solution)
princess

'P5'

In [25]:
print(f'Die Prinzessin ist im Zimmer Nummer {princess[1]}.')

Die Prinzessin ist im Zimmer Nummer 5.


Finally, we check whether the solution is unique.  If the solution is not unique, then you have missed to code some of the requirements.

In [26]:
def checkUniqueness(Clauses, princess):
    Clauses.add(frozenset({('¬', princess)}))
    alternative = solve(Clauses)
    if alternative == { frozenset() }:
        print('The solution is unique.')
    else:
        print(alternative)

In [27]:
checkUniqueness(Clauses, princess)

The solution is unique.
